In [1]:
import sys
sys.path.append('src')

In [2]:
from bm25_v2 import BM25Searcher
import pandas as pd
import numpy as np
from utils import get_combined_df, Args
from eval import SearchEvaluator, ModelEvaluator

In [3]:
data_path="data/2_7/facebook_react"
repo_name = data_path.split("/")[-1]
combined_df = get_combined_df(data_path)
index_path=f"{data_path}/index_commit_tokenized"
k=1000 # initial ranker depth
n=100 # number of samples to evaluate on

In [4]:
metrics = ['MAP', 'P@1', 'P@10', 'P@20', 'P@30', 'MRR', 'R@1', 'R@10', 'R@100', 'R@1000']
bm25_searcher = BM25Searcher(index_path)
evaluator = SearchEvaluator(metrics)
model_evaluator = ModelEvaluator(bm25_searcher, evaluator, combined_df)
gold_df = pd.read_parquet(f"gold/{repo_name}/v2_{repo_name}_gpt4_gold.parquet")

Loaded index at data/2_7/facebook_react/index_commit_tokenized
Index Stats: {'total_terms': 7587973, 'documents': 73765, 'non_empty_documents': 73765, 'unique_terms': 14602}


In [5]:
test_id, test_date, test_orig, test_files, test_query = gold_df.iloc[6]

In [6]:
test_query

"Input properties 'value' and 'defaultValue' accepting and assigning functions and symbols leads to improper handling and inconsistencies in numeric equality checks."

In [7]:
test_files

array(['packages/react-dom/src/__tests__/ReactDOMInput-test.js',
       'packages/react-dom/src/client/ReactDOMFiberInput.js',
       'packages/react-dom/src/events/ChangeEventPlugin.js',
       'packages/react-dom/src/shared/DOMProperty.js'], dtype=object)

In [9]:
bm25_results = bm25_searcher.search(test_query, test_date, k)

In [10]:
aggregated_results = bm25_searcher.aggregate_file_scores(bm25_results, 'avgp', sort_contributing_result_by_date=True)

In [11]:
sump_results = model_evaluator.evaluate_df(gold_df, 1000, aggregation_strategy='sump', rerankers=[])

100%|██████████| 100/100 [00:26<00:00,  3.77it/s]


In [12]:
recentp_results = model_evaluator.evaluate_df(gold_df, 1000, aggregation_strategy='recentp', rerankers=[])

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:26<00:00,  3.83it/s]


In [13]:
def get_avg(results):
    return {metric: round(np.mean([result[metric] for result in results]), 4) for metric in results[0]}

{'MAP': 0.1552,
 'P@1': 0.18,
 'P@10': 0.057,
 'P@20': 0.0425,
 'P@30': 0.0363,
 'MRR': 0.2331,
 'R@1': 0.0619,
 'R@10': 0.146,
 'R@100': 0.4004,
 'R@1000': 0.5801}

In [3]:
def tmp():
    args = Args(index_path='data/2_7/facebook_react/index_commit_tokenized', data_path='data/2_7/facebook_react', k=1000, n=100, no_bm25=True, model_path='microsoft/codebert-base', overwrite_cache=False, batch_size=32, num_epochs=20, learning_rate=5e-05, run_name='combined_bce_train', num_positives=10, num_negatives=10, train_depth=1000, num_workers=8, train_commits=1000, psg_cnt=5, aggregation_strategy='sump', use_gpu=True, rerank_depth=250, do_train=False, do_eval=True, eval_gold=True, openai_model='gpt4', overwrite_eval=False, sanity_check=False, debug=False, do_combined_train=False, repo_paths=['data/2_7/apache_spark', 'data/2_7/apache_kafka', 'data/2_7/facebook_react', 'data/2_8/angular_angular', 'data/2_8/django_django', 'data/2_8/pytorch_pytorch', 'data/2_7/julialang_julia', 'data/2_7/ruby_ruby', 'data/2_9/huggingface_transformers', 'data/2_9/redis_redis'], best_model_path=None, ignore_gold_in_training=False, use_gpt_train=True, eval_folder='combined_bce_train', notes='combined train with BCE', train_mode='classification')
    from BERTReranker_v4 import main
    main(args)

tmp()

Available devices:  1
Current cuda device:  0
Quadro RTX 6000
Loaded index at data/2_7/facebook_react/index_commit_tokenized
Index Stats: {'total_terms': 7587973, 'documents': 73765, 'non_empty_documents': 73765, 'unique_terms': 14602}
Using classification model


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using device: cuda
Using GPU: Quadro RTX 6000
GPU Device Count: 1
GPU Memory Usage: 476.73 MB
Initialized BERT reranker with parameters: {'model_name': 'microsoft/codebert-base', 'psg_cnt': 25, 'aggregation_strategy': 'maxp', 'batch_size': 32, 'use_gpu': True, 'rerank_depth': 100, 'num_epochs': 20, 'lr': 5e-05, 'num_positives': 10, 'num_negatives': 10, 'train_depth': 1000, 'num_workers': 8, 'train_commits': 1500, 'bm25_aggr_strategy': 'sump'}


AttributeError: 'Args' object has no attribute 'do_combined_train'